In [1]:
import os

import numpy as np
import pandas as pd
from tslearn.metrics import dtw, dtw_path

import tensorflow as tf
#from tensorflow.keras import backend as k
from tensorflow import keras

#from keras import backend
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Input, Add, Activation, Dropout,Reshape

def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg,num_features):
    def f(input_):
        
        #input = [batch_size,input_seqlength,num_features] (N,9,7) 
        residual= input_        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False,
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(input_)
                    
        layer_out =   Activation('selu')(layer_out)
        
        skip_out =    Conv1D(num_features,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
        
        network_in =  Conv1D(num_features,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
                      
        network_out = Add()([residual, network_in])
        
        return network_out, skip_out
    
    return f

def DC_CNN_Model(input_seq_length,num_features): #length [6], num_features[7]
    input = Input(shape=(input_seq_length,num_features))
    l1a, l1b = DC_CNN_Block(32,2,1,0.001,num_features)(input)   #listo
    l2a, l2b = DC_CNN_Block(32,2,2,0.001,num_features)(l1a)
    l3a, l3b = DC_CNN_Block(32,2,4,0.001,num_features)(l2a)
    l4a, l4b = DC_CNN_Block(32,2,8,0.001,num_features)(l3a)
    l5a, l5b = DC_CNN_Block(32,2,16,0.001,num_features)(l4a)
    l6a, l6b = DC_CNN_Block(32,2,32,0.001,num_features)(l5a)
    l6b = Dropout(0.8)(l6b) #dropout used to limit influence of earlier data
    l7a, l7b = DC_CNN_Block(32,2,64,0.001,num_features)(l6a)
    l7b = Dropout(0.8)(l7b) #dropout used to limit influence of earlier data
    l8 =   Add()([l1b, l2b, l3b, l4b, l5b, l6b, l7b])
    l9 =   Activation('relu')(l8)
    
    kernel_size = 7 if input_seq_length ==9 else 1
    l21 =  Conv1D(num_features,kernel_size, activation='linear', use_bias=False, 
           kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, seed=42),
           kernel_regularizer=l2(0.001))(l9)

    model = keras.Model(inputs=input, outputs=l21)
    adam = optimizers.Adam(lr=0.00075,beta_1=0.9, beta_2=0.999,decay=0.0,)
    model.compile(loss=rmse,optimizer=adam,metrics=[rmse,'mae'])

    return model

def forecasting_dcnn(train_seqs,train_targets,val_seqs,val_targets,
                     input_seq_length,output_seq_length,num_features,
                     num_epochs,batch_size,path):

    model = DC_CNN_Model(input_seq_length,num_features)
    print('Model: input size {}, output size {}'.format(model.input_shape, model.output_shape))

    path_checkpoint = path+'model/'+'mybest_model.h5'
    es_callback = EarlyStopping(monitor="val_rmse",mode='min', patience=50)
    modelckpt_callback = ModelCheckpoint(monitor="val_rmse",filepath=path_checkpoint,verbose=1,save_weights_only=True,save_best_only=True,mode='min')
    
    validation_set = (val_seqs,val_targets)
    history = model.fit(train_seqs,train_targets,validation_data = validation_set,
                        epochs=num_epochs, batch_size=batch_size,verbose=1,
                        callbacks=[es_callback, modelckpt_callback])   
    history = history.history
    return history, model

2022-05-19 18:27:36.307873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
conv1 = Conv1D(filters=32, kernel_size=2, 
                dilation_rate=1, 
                activation='linear', padding='causal', use_bias=False,
                kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                seed=42), kernel_regularizer=l2(0.001))

In [3]:
tensor = tf.random.uniform([1,6,7]) #[batch_size,input_seqlength,num_features]
print(tensor)

tf.Tensor(
[[[0.29474843 0.20440185 0.92647827 0.50443614 0.81025016 0.6354084
   0.5656383 ]
  [0.1691662  0.45696235 0.02349174 0.05894935 0.23700058 0.7668153
   0.6083363 ]
  [0.8155527  0.491179   0.13215053 0.9824059  0.49891913 0.93679357
   0.16109443]
  [0.9108881  0.9186382  0.7215165  0.02761078 0.5015073  0.1631118
   0.1743511 ]
  [0.18427217 0.2595849  0.57357    0.4241433  0.94633067 0.60525894
   0.42312884]
  [0.91149366 0.12254357 0.3543806  0.8583298  0.08962643 0.8000027
   0.14847088]]], shape=(1, 6, 7), dtype=float32)


2022-05-19 18:27:37.666190: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-19 18:27:37.668469: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-19 18:27:37.732378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 18:27:37.733347: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 with Max-Q Design computeCapability: 6.1
coreClock: 1.48GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-05-19 18:27:37.733440: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-19 18:27:37.741305: I tensorflow/stream_exe

In [4]:
conv1(tensor).shape

TensorShape([1, 6, 32])